In [1]:
# このノートブックでは、baselineとの違いを分かりやすくするため、変更した点にのみ注釈を付けている

import pandas as pd
train = pd.read_csv("../../data/kaggle/titanic/train.csv")
test = pd.read_csv("../../data/kaggle/titanic/test.csv")
gender_submission = pd.read_csv("../../data/kaggle/titanic/gender_submission.csv")

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
print(len(train))
train.isna().sum()

891


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
age_median = train.groupby(["Sex","Pclass"])["Age"].median()
age_median
# baselineではAgeの欠損を単に中央値で補完していたが、groupbyを使用し、性別、階級ごとのグループでより正確な数値で補完

Sex     Pclass
female  1         35.0
        2         28.0
        3         21.5
male    1         40.0
        2         30.0
        3         25.0
Name: Age, dtype: float64

In [8]:
# baselineではAgeの欠損を単一の中央値で補完していたが、
# 今回はSex,Pclassごとに異なる中央値を用いるため、行ごとに条件分岐が必要

# fillna()は単純な処理には向いてるが、今回のような条件付き補完には不向き
# よって、関数を定義し、for文とmaskを用いて補完を行う

def fill_age_df(df):
    for sex in ["male", "female"]:
        for pclass in [1,2,3]:
            mask = (df["Sex"] == sex) & (df["Pclass"] == pclass) & (df["Age"].isna())
            df.loc[mask, "Age"] = age_median[sex, pclass]
    return df
# 最初のfor文１回転につき、２番目のfor文が３回転し、合計で最初のfor文が２回転、２番目のfor文が６回転

# maskの意味は、今見ている((df["Sex"] == sex) & (df["Pclass"] == pclass))かつ、
# Ageが欠損している(df["Age"].isna())という条件を満たすdf内の行
# .loc[条件,指定の列]

train = fill_age_df(train)

train = train.dropna(subset=["Embarked"])
print(len(train))
train.isna().sum()

889


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [9]:
train = train[["Survived","Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
train = pd.get_dummies(train,dtype=int)
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [10]:
del train["Sex_male"]
del train["Embarked_S"]
train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q
0,0,3,22.0,1,0,7.2500,0,0,0
1,1,1,38.0,1,0,71.2833,1,1,0
2,1,3,26.0,0,0,7.9250,1,0,0
3,1,1,35.0,1,0,53.1000,1,0,0
4,0,3,35.0,0,0,8.0500,0,0,0


In [11]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

X = train
y = train["Survived"]
del X["Survived"]

X_train,X_test,y_train,y_test = sklearn.model_selection.train_test_split(X,y,random_state=0)
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train,y_train)

print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

0.726457399103139
0.984984984984985


In [ ]:
model = DecisionTreeClassifier(random_state=0,max_depth=6,min_samples_leaf=10)
# 決定木の深さ（max_depth）だけでなく、
# 葉に含まれる最小サンプル数（min_samples_leaf）も指定することで、
# 過度に細かい分岐を抑え、更に過学習を防ぐ


model.fit(X_train, y_train)

print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.7847533632286996
0.8753753753753754


In [13]:
test_raw = test.copy()

age_median_test = test_raw.groupby(["Sex","Pclass"])["Age"].median()
test_raw = fill_age_df(test_raw)

fare_median_test = test_raw["Fare"].median()
test_raw["Fare"] = test_raw["Fare"].fillna(fare_median_test)

test_raw = test_raw[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
test_raw = pd.get_dummies(test_raw,dtype=int)
test_raw = test_raw.reindex(columns=X.columns, fill_value=0)
test_raw.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q
0,3,34.5,0,0,7.8292,0,0,1
1,3,47.0,1,0,7.0000,1,0,0
2,2,62.0,0,0,9.6875,0,0,1
3,3,27.0,0,0,8.6625,0,0,0
4,3,22.0,1,1,12.2875,1,0,0


In [14]:
pred = model.predict(test_raw)

In [16]:
submission_improved = pd.DataFrame({"PassengerId": test["PassengerId"],"Survived": pred})
submission_improved.to_csv("../../outputs/kaggle_titanic/submission_improved.csv", index=False)